In [ ]:
#instalacja i importowanie potrzebnych bibliotek
!pip install fastbook
import fastbook
from fastbook import *
from fastai.vision.widgets import *
from IPython.core.pylabtools import figsize

In [ ]:
#łączenie się z dyskiem googla i pobieranie odpowiednich obrazów z sieci
#pobrane obrazy należy ręcznie przefiltrować (usunąć błędy w pobranych obrazach lub nieodpowiednie dla kategorii zdjęcia)
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dataSet')
path = Path('/content/drive/My Drive/dataSet')

klasy = 'Wolf', 'Lynx', 'Moose', 'Deer', 'Boar'
path = Path('Projekt_PUM_1')
if not path.exists(): 
  path.mkdir()
  for o in klasy:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    urls = search_images_ddg(f' {o} animals', max_images=110)
    download_images(dest, urls=urls)


In [ ]:
classes = ['Wolf', 'Lynx', 'Moose', 'Deer', 'Boar']

In [ ]:
#tworzenie DataBlock
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [ ]:
#Batch size = 32, num_workers = 0
data = data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(max_rotate=2.0, p_affine=0.5)
)
dls = data.dataloaders(path, bs = 32, num_workers = 0)

In [ ]:
#tworzenie konwolucyjnej sieci neuronowej oraz określanie architekruty, w tym przypadku 'alexnet'
learn = vision_learner(dls, alexnet, metrics=accuracy)

In [ ]:
#trenowanie modelu, 10 epok, earlystopping, patience = 5
#ustawianie najlepszego zakresu dla learning rate
learn.fit(10, cbs = [SaveModelCallback(), EarlyStoppingCallback(patience=5)])
learn.save('stage-1')

learn.lr_find()

learn.fit(10, 10e-1, cbs = [SaveModelCallback(), EarlyStoppingCallback(patience=5)])
learn.save('stage-2') 

In [ ]:
#exportowanie modelu
learn.export(fname='model.pkl')